### 2.  Голосовой ассистент
•  Персональный менеджер клиента ведёт диалог с состоятельным клиентом в системе Дион

•  В течении диалога персонального менеджера с клиентом ассистент должен мониторить ход беседы включая тональность диалога и корректность следования целям встречи, подсказывая, если необходимо сменить направление или тональность диалога

•  Если в процессе диалога с клиентом персональный менеджер получает вопрос, на который затрудняется ответить, ассистент должен сформировать для него подсказку по проблеме, с которой он столкнулся

•  По результатам диалога с клиентом необходимо сформировать самари прошедшей встречи для отправки в операционный CRM

---

**Решение:**

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

ORGANIZATION = os.environ.get("ORGANIZATION")
PROJECT = os.environ.get("PROJECT")
APIKEY = os.environ.get("APIKEY")

Используем 4o-mini как MVP.

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
import torch
import threading
from openai import OpenAI

client = OpenAI(
    organization=ORGANIZATION,
    project=PROJECT,
    api_key=APIKEY
)
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo", device=0 if torch.cuda.is_available() else -1)

Здесь - сделаем 3 функции с промптами - для разбиения диалога по ролям; для оценки и рекомендаций менеджеру; для формирования саммари.

Скрипт сгенерирован синтетически, как и диалоги (с отклонениями от скрипта, тональности, так и без - озвучены в elevenlabs).

In [2]:
def send_to_llm_split(dialog):

  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    # намеренно внесены опечатки в транскрибацию
    messages=[
      {"role": "system", "content": """Разбей диалог, на менеджера и клиента вида:
Менеджер: ...
Клиент: ...
Менеджер: ...
Клиент: ...
...

Менеджер предлагает услуги банка ВТБ и задает вопросы, а клиент на них отвечает.
       
Пример разбиения диалога:
<НАЧАЛО ПРИМЕРА>
Транскрибация:
Добрый день, Сергей, меня зовут Роман, я представляю Банка ТБ. Удобно ли вам сейчас говорить? Да, у меня есть несколько минут. Отлично, мы выходим. о нашей новой инвестиционной программе, разработанной специально для состоятельных клиентов. Эта программа ориентирована на сохранение и примножение капитала с высокой степенью защиты и прогнозируемой доходностью. Скажите, какие Какие стратегии вам наиболее интересны? Долгосрочные консервативные вложения, сбалансированные портфели или, возможно, венчурные инвестиции? Я предпочитаю долгосрочные вложения с умеренным выбор. В нашей программе мы предлагаем индивидуальное управление активами с экспертным сопровождением, доступ к эксклюзивным инвестиционным инструментам и оптимизированной налоговой стратегии. Позвольте уточнить, что такое для вас сейчас приоритетен? Сохранение капитала, его рост или максимизация ликвидности? В первую очередь, сохранение, но с возможностью хорошей доходности. Тогда я бы давал вам рассмотреть портфель с низкой волатильностью, включающей облигации, цены бумаги, с фиксированной доходностью и хедж-онды. Можем организовать встречу с нашим инвестиционным консультантом, В какое время вам было бы удобно обсудить это подробнее? Давайте на следующей неделе во вторник после обеда. Отлично, во вторник после обеда вы позвоню, чтобы согласовать детали. Спасибо за ваш интерес, Сергей, и хорошего вам день.

Результат:
Менеджер: Добрый день, Сергей, меня зовут Роман, я представляю Банк ВТБ. Удобно ли вам сейчас говорить?
Клиент: Да, у меня есть несколько минут.
Менеджер: Отлично, мы выходим с информацией о нашей новой инвестиционной программе, разработанной специально для состоятельных клиентов. Эта программа ориентирована на сохранение и примножение капитала с высокой степенью защиты и прогнозируемой доходностью. Скажите, какие стратегии вам наиболее интересны? Долгосрочные консервативные вложения, сбалансированные портфели или, возможно, венчурные инвестиции?
Клиент: Я предпочитаю долгосрочные вложения с умеренным риском.
Менеджер: В нашей программе мы предлагаем индивидуальное управление активами с экспертным сопровождением, доступ к эксклюзивным инвестиционным инструментам и оптимизированной налоговой стратегии. Позвольте уточнить, что для вас сейчас приоритетнее? Сохранение капитала, его рост или максимизация ликвидности?
Клиент: В первую очередь, сохранение капитала, но с возможностью хорошей доходности.
Менеджер: Тогда я бы предложил вам рассмотреть портфель с низкой волатильностью, включающий облигации, ценные бумаги с фиксированной доходностью и хедж-фонды. Можем организовать встречу с нашим инвестиционным консультантом. В какое время вам было бы удобно обсудить это подробнее?
Клиент: Давайте на следующей неделе во вторник после обеда.
Менеджер: Отлично, во вторник после обеда я вам позвоню, чтобы согласовать детали. Спасибо за ваш интерес, Сергей, и хорошего вам дня.
<КОНЕЦ ПРИМЕРА>

"""},
      {"role": "user", "content": "Вот транскрибация диалога между менеджером и клиентом:\n" + dialog}
    ],
    temperature=0.7,
  )

  return completion.choices[0].message.content

def send_to_llm_check(script, dialog):

  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": f"""Дан диалог менеджера банка ВТБ с состоятельным клиентом.
       
Вот скрипт, которому должен соответствовать менеджер:
{script}

Необходимо вернуть решение в формате JSON. 
Тональность последних реплик менеджера (профессиональный и уважительный тон) - "tonality" может принимать значения "ok", либо "not ok", 
Рекомендации на текущий момент - "recommendations" - краткая подсказка, о чем говорить дальше менеджеру согласно скрипту и, возможно, сменить тональность диалога.
Если менеджер отклоняется от скрипта, напишите об этом в "recommendations" JSON.

Пример JSON:
{{"tonality": "ok", "recommendations": "Перейдите к обсуждению преференций клиента касаемо сферы инвестиций."}}
"""},
      {"role": "user", "content": "Вот диалог между менеджером и клиентом:\n" + dialog}
    ],
    temperature=0.7,
  )

  return completion.choices[0].message.content.strip("```json")

def send_to_llm_summary(script, dialog):

  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": f"""Дан диалог менеджера банка ВТБ с состоятельным клиентом.
       
Вот скрипт, которому должен соответствовать менеджер:
{script}

Сформируй саммари (подробные итоги) по диалогу менеджера с клиентом и укажи, соблюдался ли полностью скрипт и уважительная тональность. 
Не используй Markdown.
"""},
      {"role": "user", "content": "Вот диалог между менеджером и клиентом:\n" + dialog}
    ],
    temperature=0.7,
  )

  return completion.choices[0].message.content

SCRIPT = """
Добрый день, [Имя клиента]. Меня зовут [Ваше имя], я представляю банк ВТБ. Удобно ли вам сейчас говорить?

Я хочу поблагодарить вас за то, что выбрали наш банк. Мы всегда стремимся предложить нашим клиентам только лучшие решения, и я бы хотел рассказать вам о наших новых инвестиционных продуктах, которые могут быть интересны именно вам.

Наши специалисты разработали уникальную инвестиционную программу, которая ориентирована на клиентов с высоким капиталом, таких как вы. Программа включает в себя разнообразные стратегии, направленные на сохранение и приумножение активов, с высокой степенью защиты и прогнозируемой доходностью. Все решения разрабатываются с учетом ваших потребностей и финансовых целей.

Скажите, какие из следующих инвестиционных стратегий вам наиболее интересны?

    Долгосрочные консервативные вложения: эта стратегия подойдет тем, кто предпочитает минимальные риски и хочет сохранить капитал с прогнозируемым доходом.
    Диверсифицированные портфели: сбалансированное сочетание различных активов, которые помогут вам не только защитить капитал, но и получать стабильную доходность.
    Эксклюзивные возможности в венчурных или альтернативных инвестициях: для тех, кто ищет более высокую доходность и готов к умеренным рискам.

Немного о преимуществах наших продуктов:

    Индивидуальное управление активами с экспертным сопровождением. Мы анализируем ваш финансовый портфель, учитываем цели и предпочтения, чтобы предложить наиболее эффективное решение.
    Доступ к закрытым инвестиционным возможностям. Это могут быть уникальные предложения, доступные только для избранных клиентов, что позволяет значительно увеличить доходность вашего портфеля.
    Оптимизированные стратегии налогообложения. Наши специалисты помогут вам минимизировать налоговые риски и выбрать самые выгодные для вас инструменты.

Позвольте уточнить, какие у вас сейчас приоритеты в управлении капиталом? Вы заинтересованы в стратегиях, ориентированных на сохранение капитала, рост активов или, возможно, вы ищете варианты для максимизации ликвидности и быстрого доступа к деньгам в случае необходимости?

Если вам интересно, мы можем предложить вам персонализированную инвестиционную модель, основанную на вашем профиле риска и долгосрочных целях. Это позволит вам выбрать наиболее подходящие инвестиционные инструменты и стратегии. Хотите, я расскажу подробнее о конкретных продуктах, которые могут быть вам полезны?

Помимо этого, мы можем организовать встречу с нашим ведущим инвестиционным консультантом, который поможет вам выбрать наиболее подходящие решения и ответит на все ваши вопросы. Мы можем провести встречу в удобное для вас время — пожалуйста, сообщите, когда вам будет комфортно.

Если сейчас не самый удобный момент, мы можем назначить удобное время для звонка или встречи. В какое время вам было бы удобно обсудить это более подробно?

Я понимаю, что финансовые решения требуют времени и внимания, поэтому я готов помочь вам на каждом этапе этого пути. Заранее благодарю за ваше время и внимание, и буду рад обсудить все детали в любое удобное для вас время.
"""

Интерфейс в Gradio (как на скрине в README):

In [ ]:
transcribed_text, fixed_text_llm, recs_text_llm = "", "", ""
audio_buffer = np.array([])

TIMING = 10

def process_text(text):
    global transcribed_text
    transcribed_text += text

def process_text_llm():
    global transcribed_text, fixed_text_llm, recs_text_llm
    if transcribed_text:
        fixed_text_llm = send_to_llm_split(transcribed_text)
        recs_text_llm = send_to_llm_check(SCRIPT, fixed_text_llm)
    threading.Timer(TIMING, process_text_llm).start()

process_text_llm()

def transcribe(stream, new_chunk):
    global audio_buffer
    sr, y = new_chunk

    if y.ndim > 1:
        y = y.mean(axis=1)

    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    audio_buffer = np.concatenate([audio_buffer, y])
    transcribed = ""

    if audio_buffer.shape[0] >= sr * TIMING:
        audio_chunk = audio_buffer[:sr * TIMING]
        transcribed = transcriber({"sampling_rate": sr, "raw": audio_chunk, "language": "russian"})["text"]
        
        threading.Thread(target=process_text, args=(transcribed,)).start()

        audio_buffer = audio_buffer[sr * TIMING:]

    return stream, fixed_text_llm, recs_text_llm

def clear_buffers():
    global transcribed_text, fixed_text_llm, recs_text_llm, audio_buffer
    transcribed_text, fixed_text_llm, recs_text_llm = "", "", ""
    audio_buffer = np.array([])
    return "", "", ""

def generate_summary():
    global transcribed_text, fixed_text_llm, recs_text_llm, audio_buffer
    
    if not transcribed_text:
        return "Нет данных для саммари."
    
    summary = send_to_llm_summary(SCRIPT, fixed_text_llm)
    
    transcribed_text, fixed_text_llm, recs_text_llm = "", "", ""
    audio_buffer = np.array([])
    
    return summary

with gr.Blocks(css="""
    #component-14, #component-17 {
        display: none !important;
    }
""") as demo:
    gr.Markdown("# Задача 2. LLM-based голосовой ассистент")
    audio_input = gr.Audio(sources=["microphone"], streaming=True)
    dialog_box = gr.Textbox(label="Диалог:")
    recs_box = gr.Textbox(label="Рекомендации менеджеру:")
    summary_button = gr.Button("Сформулировать саммари")
    clear_button = gr.Button("Начать заново")
    summary_box = gr.Textbox(label="Саммари:", interactive=False)
    
    demo_interface = gr.Interface(
        transcribe,
        ["state", audio_input],
        ["state", dialog_box, recs_box], 
        live=True,
    )
    clear_button.click(clear_buffers, inputs=[], outputs=[dialog_box, recs_box, summary_box])
    summary_button.click(generate_summary, inputs=[], outputs=[summary_box])

demo.launch()